# Comparación de Métodos SARSA Semi-gradiente en MountainCar

Este notebook compara tres implementaciones de SARSA semi-gradiente con diferentes representaciones de características:
1. **Fourier Basis Functions**
2. **Tile Coding**
3. **Radial Basis Functions (RBF)**

In [2]:
!pip install gymnasium
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\dario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\dario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import random
from itertools import product
from concurrent.futures import ProcessPoolExecutor, as_completed
import imageio
from IPython.display import HTML
import seaborn as sns
import time

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# -------------------- IMPLEMENTACIONES DE LOS AGENTES --------------------

# Agente Fourier
class AgenteSARSASemiGradienteFourier:
    def __init__(self, env, alpha=0.01, gamma=1.0, epsilon=1.0, decay=True, fourier_order=3):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay = decay
        self.fourier_order = fourier_order
        self.name = "Fourier"

        self.nA = env.action_space.n
        self.low = env.observation_space.low
        self.high = env.observation_space.high
        self.state_dim = env.observation_space.shape[0]

        self.c = np.array(list(product(range(fourier_order + 1), repeat=self.state_dim)))
        self.d = len(self.c)
        self.theta = np.zeros((self.nA, self.d))
        self.stats = []
        self.episode_lengths = []

    def _scale_state(self, state):
        return (state - self.low) / (self.high - self.low)

    def _phi(self, state):
        s_scaled = self._scale_state(state)
        return np.cos(np.pi * np.dot(self.c, s_scaled))

    def _Q(self, state, action):
        return np.dot(self.theta[action], self._phi(state))

    def _seleccionar_accion(self, state):
        policy = self._epsilon_soft_policy(state)
        return np.random.choice(np.arange(self.nA), p=policy)

    def _epsilon_soft_policy(self, state):
        q_values = np.array([self._Q(state, a) for a in range(self.nA)])
        policy = np.ones(self.nA) * self.epsilon / self.nA
        best_action = np.argmax(q_values)
        policy[best_action] += 1.0 - self.epsilon
        return policy

    def entrenar(self, num_episodes=5000, mostrar_barra=False):
        acumulador_recompensas = 0.0
        for t in tqdm(range(num_episodes), disable=not mostrar_barra, desc=f"Entrenando {self.name}"):
            if self.decay:
                self.epsilon = min(1.0, 1000.0 / (t + 1))
            
            state, _ = self.env.reset()
            action = self._seleccionar_accion(state)
            done = False
            total_reward = 0
            pasos = 0

            while not done:
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                next_action = self._seleccionar_accion(next_state)

                phi = self._phi(state)
                if np.any(np.isnan(phi)) or np.any(np.isinf(phi)):
                    phi = np.nan_to_num(phi)

                q_current = self._Q(state, action)
                q_next = self._Q(next_state, next_action) if not done else 0.0
                target = reward + self.gamma * q_next
                delta = target - q_current

                if np.isnan(delta) or np.isinf(delta):
                    delta = 0.0

                self.theta[action] += self.alpha * delta * phi
                self.theta[action] = np.clip(self.theta[action], -1e3, 1e3)

                state = next_state
                action = next_action
                total_reward += reward
                pasos += 1

            self.episode_lengths.append(pasos)
            acumulador_recompensas += total_reward
            self.stats.append(acumulador_recompensas / (t + 1))

        return self.theta

# Agente Tile Coding
class TileCoder:
    def __init__(self, low, high, bins=(10, 10), num_tilings=8):
        self.low = np.array(low)
        self.high = np.array(high)
        self.bins = np.array(bins)
        self.num_tilings = num_tilings
        self.tile_width = (self.high - self.low) / (self.bins - 1)
        self.total_tiles = np.prod(self.bins) * num_tilings
        self.offsets = [
            np.random.uniform(0, self.tile_width, size=len(self.low))
            for _ in range(num_tilings)
        ]

    def get_features(self, state):
        features = np.zeros(self.total_tiles)
        for i, offset in enumerate(self.offsets):
            shifted = np.array(state) + offset
            idx = ((shifted - self.low) / self.tile_width).astype(int)
            idx = np.clip(idx, 0, self.bins - 1)
            tile_index = np.ravel_multi_index(idx, self.bins)
            feature_index = i * np.prod(self.bins) + tile_index
            features[int(feature_index)] = 1
        return features

class AgenteSARSATileCoding:
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=1.0, decay=True, bins=(8, 8), num_tilings=8):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay = decay
        self.name = "Tile Coding"

        self.nA = env.action_space.n
        self.tile_coder = TileCoder(env.observation_space.low, env.observation_space.high, bins, num_tilings)
        self.d = self.tile_coder.total_tiles
        self.theta = np.zeros((self.nA, self.d))
        self.stats = []
        self.episode_lengths = []

    def _Q(self, state, action):
        phi = self.tile_coder.get_features(state)
        return np.dot(self.theta[action], phi)

    def _seleccionar_accion(self, state):
        policy = self._epsilon_soft_policy(state)
        return np.random.choice(np.arange(self.nA), p=policy)

    def _epsilon_soft_policy(self, state):
        q_values = np.array([self._Q(state, a) for a in range(self.nA)])
        if np.any(np.isnan(q_values)) or np.any(np.isinf(q_values)):
            q_values = np.nan_to_num(q_values)
        
        policy = np.ones(self.nA) * self.epsilon / self.nA
        best_action = np.argmax(q_values)
        policy[best_action] += 1.0 - self.epsilon
        
        policy = np.clip(policy, 0, 1)
        if policy.sum() == 0:
            policy = np.ones(self.nA) / self.nA
        else:
            policy /= policy.sum()
        
        return policy

    def entrenar(self, num_episodes=5000, mostrar_barra=False):
        acumulador_recompensas = 0.0
        for t in tqdm(range(num_episodes), disable=not mostrar_barra, desc=f"Entrenando {self.name}"):
            if self.decay:
                self.epsilon = max(0.05, 1000.0 / (t + 1))

            state, _ = self.env.reset(seed=42)
            action = self._seleccionar_accion(state)
            done = False
            total_reward = 0
            pasos = 0

            while not done:
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                next_action = self._seleccionar_accion(next_state)

                phi = self.tile_coder.get_features(state)
                q_next = self._Q(next_state, next_action) if not done else 0.0
                q_current = np.dot(self.theta[action], phi)
                target = reward + self.gamma * q_next
                delta = target - q_current

                if not (np.isnan(delta) or np.isinf(delta)):
                    self.theta[action] += self.alpha * delta * phi
                    self.theta[action] = np.clip(self.theta[action], -1e6, 1e6)

                state = next_state
                action = next_action
                total_reward += reward
                pasos += 1

            self.episode_lengths.append(pasos)
            acumulador_recompensas += total_reward
            self.stats.append(acumulador_recompensas / (t + 1))

        return self.theta

# Agente RBF
class RBFBasisFunctions:
    def __init__(self, low, high, num_centers=(5, 5), sigma=0.2):
        self.low = np.array(low)
        self.high = np.array(high)
        self.num_centers = np.array(num_centers)
        self.sigma = sigma
        
        pos_centers = np.linspace(low[0], high[0], num_centers[0])
        vel_centers = np.linspace(low[1], high[1], num_centers[1])
        
        self.centers = []
        for p in pos_centers:
            for v in vel_centers:
                self.centers.append([p, v])
        
        self.centers = np.array(self.centers)
        self.num_features = len(self.centers)
    
    def get_features(self, state):
        state = np.array(state)
        features = np.zeros(self.num_features)
        
        for i, center in enumerate(self.centers):
            distance = np.linalg.norm(state - center)
            features[i] = np.exp(-(distance ** 2) / (2 * self.sigma ** 2))
        
        return features

class AgenteSARSARBF:
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=1.0, decay=True, num_centers=(5, 5), sigma=0.2):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay = decay
        self.name = "RBF"

        self.nA = env.action_space.n
        self.low = env.observation_space.low
        self.high = env.observation_space.high
        self.state_dim = env.observation_space.shape[0]

        self.rbf = RBFBasisFunctions(self.low, self.high, num_centers, sigma)
        self.d = self.rbf.num_features
        self.theta = np.zeros((self.nA, self.d))
        self.stats = []
        self.episode_lengths = []

    def _phi(self, state):
        return self.rbf.get_features(state)

    def _Q(self, state, action):
        return np.dot(self.theta[action], self._phi(state))

    def _seleccionar_accion(self, state):
        policy = self._epsilon_soft_policy(state)
        return np.random.choice(np.arange(self.nA), p=policy)

    def _epsilon_soft_policy(self, state):
        q_values = np.array([self._Q(state, a) for a in range(self.nA)])
        
        if np.any(np.isnan(q_values)) or np.any(np.isinf(q_values)):
            q_values = np.nan_to_num(q_values)
            
        policy = np.ones(self.nA) * self.epsilon / self.nA
        best_action = np.argmax(q_values)
        policy[best_action] += 1.0 - self.epsilon
        
        policy = np.clip(policy, 0, 1)
        if policy.sum() == 0:
            policy = np.ones(self.nA) / self.nA
        else:
            policy /= policy.sum()
            
        return policy

    def entrenar(self, num_episodes=5000, mostrar_barra=False):
        acumulador_recompensas = 0.0
        for t in tqdm(range(num_episodes), disable=not mostrar_barra, desc=f"Entrenando {self.name}"):
            if self.decay:
                self.epsilon = min(1.0, 1000.0 / (t + 1))
            
            state, _ = self.env.reset()
            action = self._seleccionar_accion(state)
            done = False
            total_reward = 0
            pasos = 0

            while not done:
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                next_action = self._seleccionar_accion(next_state)

                phi = self._phi(state)
                if np.any(np.isnan(phi)) or np.any(np.isinf(phi)):
                    phi = np.nan_to_num(phi)

                q_current = self._Q(state, action)
                q_next = self._Q(next_state, next_action) if not done else 0.0
                target = reward + self.gamma * q_next
                delta = target - q_current

                if np.isnan(delta) or np.isinf(delta):
                    delta = 0.0

                self.theta[action] += self.alpha * delta * phi
                self.theta[action] = np.clip(self.theta[action], -1e3, 1e3)

                state = next_state
                action = next_action
                total_reward += reward
                pasos += 1

            self.episode_lengths.append(pasos)
            acumulador_recompensas += total_reward
            self.stats.append(acumulador_recompensas / (t + 1))

        return self.theta

# Comparación de Rendimiento

In [ ]:
def comparar_agentes(num_episodes=10000, num_runs=5):
    """Compara los tres métodos ejecutando múltiples runs y calculando estadísticas."""
    
    # Configuraciones optimizadas para cada método
    configs = {
        'Fourier': {'alpha': 0.01, 'gamma': 1.0, 'epsilon': 0.3, 'fourier_order': 5, 'decay': False},
        'Tile Coding': {'alpha': 0.01, 'gamma': 1.0, 'epsilon': 0.1, 'decay': False, 'bins': (8, 8), 'num_tilings': 8},
        'RBF': {'alpha': 0.1, 'gamma': 1.0, 'epsilon': 0.3, 'num_centers': (7, 7), 'sigma': 0.2, 'decay': False}
    }
    
    resultados = {metodo: {'recompensas': [], 'longitudes': [], 'tiempos': []} for metodo in configs.keys()}
    
    for run in range(num_runs):
        print(f"\n🔄 Ejecutando run {run + 1}/{num_runs}")
        
        for metodo, config in configs.items():
            print(f"  Entrenando {metodo}...")
            env = gym.make("MountainCar-v0")
            env.reset(seed=SEED + run)
            
            start_time = time.time()
            
            if metodo == 'Fourier':
                agente = AgenteSARSASemiGradienteFourier(env, **config)
            elif metodo == 'Tile Coding':
                agente = AgenteSARSATileCoding(env, **config)
            elif metodo == 'RBF':
                agente = AgenteSARSARBF(env, **config)
            
            agente.entrenar(num_episodes=num_episodes, mostrar_barra=True)
            
            end_time = time.time()
            
            resultados[metodo]['recompensas'].append(agente.stats)
            resultados[metodo]['longitudes'].append(agente.episode_lengths)
            resultados[metodo]['tiempos'].append(end_time - start_time)
            
            env.close()
    
    return resultados

# Ejecutar comparación
print("🚀 Iniciando comparación de métodos SARSA Semi-gradiente...")
resultados = comparar_agentes(num_episodes=8000, num_runs=3)

🚀 Iniciando comparación de métodos SARSA Semi-gradiente...

🔄 Ejecutando run 1/3
  Entrenando Fourier...


Entrenando Fourier: 100%|██████████| 8000/8000 [09:46<00:00, 13.63it/s]


  Entrenando Tile Coding...


Entrenando Tile Coding: 100%|██████████| 8000/8000 [1:01:50<00:00,  2.16it/s]



  Entrenando RBF...


Entrenando RBF:  21%|██▏       | 1714/8000 [28:29<2:40:36,  1.53s/it]

In [ ]:
def visualizar_comparacion(resultados):
    """Visualiza los resultados de la comparación."""
    
    # 1. Recompensas promedio a lo largo del entrenamiento
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    for metodo in resultados.keys():
        recompensas_runs = resultados[metodo]['recompensas']
        recompensas_mean = np.mean(recompensas_runs, axis=0)
        recompensas_std = np.std(recompensas_runs, axis=0)
        
        episodes = range(len(recompensas_mean))
        plt.plot(episodes, recompensas_mean, label=metodo, linewidth=2)
        plt.fill_between(episodes, 
                        recompensas_mean - recompensas_std, 
                        recompensas_mean + recompensas_std, 
                        alpha=0.2)
    
    plt.title('Recompensa Media Acumulada')
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa Media')
    plt.legend()
    plt.grid(True)
    
    # 2. Longitud de episodios promedio
    plt.subplot(1, 3, 2)
    for metodo in resultados.keys():
        longitudes_runs = resultados[metodo]['longitudes']
        longitudes_mean = np.mean(longitudes_runs, axis=0)
        longitudes_std = np.std(longitudes_runs, axis=0)
        
        episodes = range(len(longitudes_mean))
        plt.plot(episodes, longitudes_mean, label=metodo, linewidth=2)
        plt.fill_between(episodes, 
                        longitudes_mean - longitudes_std, 
                        longitudes_mean + longitudes_std, 
                        alpha=0.2)
    
    plt.title('Longitud Media de Episodios')
    plt.xlabel('Episodio')
    plt.ylabel('Pasos')
    plt.legend()
    plt.grid(True)
    
    # 3. Tiempo de entrenamiento
    plt.subplot(1, 3, 3)
    metodos = list(resultados.keys())
    tiempos_mean = [np.mean(resultados[m]['tiempos']) for m in metodos]
    tiempos_std = [np.std(resultados[m]['tiempos']) for m in metodos]
    
    bars = plt.bar(metodos, tiempos_mean, yerr=tiempos_std, capsize=5, alpha=0.7)
    plt.title('Tiempo de Entrenamiento')
    plt.ylabel('Tiempo (segundos)')
    plt.xticks(rotation=45)
    
    # Añadir valores en las barras
    for bar, tiempo in zip(bars, tiempos_mean):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + tiempo*0.01, 
                f'{tiempo:.1f}s', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

visualizar_comparacion(resultados)

In [ ]:
def crear_tabla_resumen(resultados):
    """Crea una tabla resumen con las métricas de cada método."""
    
    metricas = []
    
    for metodo in resultados.keys():
        # Recompensa final promedio (últimos 100 episodios)
        recompensas_finales = [np.mean(run[-100:]) for run in resultados[metodo]['recompensas']]
        recompensa_final_mean = np.mean(recompensas_finales)
        recompensa_final_std = np.std(recompensas_finales)
        
        # Longitud final promedio (últimos 100 episodios)
        longitudes_finales = [np.mean(run[-100:]) for run in resultados[metodo]['longitudes']]
        longitud_final_mean = np.mean(longitudes_finales)
        longitud_final_std = np.std(longitudes_finales)
        
        # Tiempo promedio
        tiempo_mean = np.mean(resultados[metodo]['tiempos'])
        tiempo_std = np.std(resultados[metodo]['tiempos'])
        
        # Episodios para converger (cuando la longitud media móvil < 150)
        convergencia_episodios = []
        for run_longitudes in resultados[metodo]['longitudes']:
            ventana = 100
            medias_moviles = [np.mean(run_longitudes[i:i+ventana]) for i in range(len(run_longitudes)-ventana)]
            try:
                episodio_convergencia = next(i for i, media in enumerate(medias_moviles) if media < 150)
                convergencia_episodios.append(episodio_convergencia)
            except StopIteration:
                convergencia_episodios.append(len(run_longitudes))
        
        convergencia_mean = np.mean(convergencia_episodios)
        convergencia_std = np.std(convergencia_episodios)
        
        metricas.append({
            'Método': metodo,
            'Recompensa Final': f"{recompensa_final_mean:.2f} ± {recompensa_final_std:.2f}",
            'Longitud Final': f"{longitud_final_mean:.1f} ± {longitud_final_std:.1f}",
            'Tiempo (s)': f"{tiempo_mean:.1f} ± {tiempo_std:.1f}",
            'Episodios a Convergencia': f"{convergencia_mean:.0f} ± {convergencia_std:.0f}"
        })
    
    df = pd.DataFrame(metricas)
    return df

tabla_resumen = crear_tabla_resumen(resultados)
print("📊 Tabla Resumen de Resultados:")
print("="*80)
print(tabla_resumen.to_string(index=False))

# Análisis de Características

Vamos a analizar las características de cada método:

In [ ]:
def analizar_caracteristicas():
    """Analiza las características de cada método de aproximación."""
    
    env = gym.make("MountainCar-v0")
    
    # Crear agentes con configuraciones típicas
    agente_fourier = AgenteSARSASemiGradienteFourier(env, fourier_order=3)
    agente_tile = AgenteSARSATileCoding(env, bins=(8, 8), num_tilings=8)
    agente_rbf = AgenteSARSARBF(env, num_centers=(5, 5), sigma=0.2)
    
    # Estado de ejemplo
    state_ejemplo = np.array([-0.5, 0.0])  # posición media, velocidad cero
    
    # Obtener características
    phi_fourier = agente_fourier._phi(state_ejemplo)
    phi_tile = agente_tile.tile_coder.get_features(state_ejemplo)
    phi_rbf = agente_rbf._phi(state_ejemplo)
    
    print("🔍 Análisis de Características para estado [-0.5, 0.0]:")
    print("="*60)
    print(f"Fourier (orden {agente_fourier.fourier_order}):")
    print(f"  - Dimensionalidad: {len(phi_fourier)}")
    print(f"  - Características activas: {np.sum(phi_fourier != 0)}")
    print(f"  - Rango de valores: [{phi_fourier.min():.3f}, {phi_fourier.max():.3f}]")
    print(f"  - Características top-5: {phi_fourier[:5]}")
    
    print(f"\nTile Coding ({agente_tile.tile_coder.bins} bins, {agente_tile.tile_coder.num_tilings} tilings):")
    print(f"  - Dimensionalidad: {len(phi_tile)}")
    print(f"  - Características activas: {np.sum(phi_tile != 0)}")
    print(f"  - Características binarias: {np.all(np.isin(phi_tile, [0, 1]))}")
    print(f"  - Índices activos: {np.where(phi_tile == 1)[0]}")
    
    print(f"\nRBF ({agente_rbf.rbf.num_centers} centros, σ={agente_rbf.rbf.sigma}):")
    print(f"  - Dimensionalidad: {len(phi_rbf)}")
    print(f"  - Características activas: {np.sum(phi_rbf > 0.01)}")
    print(f"  - Rango de valores: [{phi_rbf.min():.3f}, {phi_rbf.max():.3f}]")
    print(f"  - Características top-5: {phi_rbf[:5]}")
    
    # Visualizar características
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Fourier
    axes[0].bar(range(len(phi_fourier)), phi_fourier, alpha=0.7)
    axes[0].set_title(f'Características Fourier (dim={len(phi_fourier)})')
    axes[0].set_xlabel('Índice de característica')
    axes[0].set_ylabel('Valor')
    
    # Tile Coding
    indices_activos = np.where(phi_tile == 1)[0]
    if len(indices_activos) > 0:
        axes[1].bar(indices_activos, phi_tile[indices_activos], alpha=0.7, color='orange')
    axes[1].set_title(f'Características Tile Coding (dim={len(phi_tile)})')
    axes[1].set_xlabel('Índice de característica')
    axes[1].set_ylabel('Valor')
    
    # RBF
    axes[2].bar(range(len(phi_rbf)), phi_rbf, alpha=0.7, color='green')
    axes[2].set_title(f'Características RBF (dim={len(phi_rbf)})')
    axes[2].set_xlabel('Índice de característica')
    axes[2].set_ylabel('Valor')
    
    plt.tight_layout()
    plt.show()
    
    env.close()

analizar_caracteristicas()